In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [67]:
data = pd.read_csv('/Users/ryan/Documents/01.Research Projects/18.NYC Congestion Pricing/2.Joint mode and destination modeling/(1226) choice_dataset.csv',
                   dtype = {'origin_county':'str','destination_county':'str'})

In [68]:
data.columns

Index(['origin_county', 'Pop_group', 'Trip_purpose', 'Time_period',
       'destination_county', 'origin_region', 'destination_region', 'mode',
       'Trip_num', 'auto_tt', 'transit_ivt', 'transit_at', 'transit_et',
       'transit_nt', 'non_auto_tt', 'cost', 'constant_driving',
       'constant_transit', 'constant_ondemand', 'constant_biking',
       'constant_walking', 'constant_carpool', 'market_id', 'alternative_id',
       'All_trips', 'Trip_0', 'share', 'share_0', 'log_s_s0', 'log_s_s(mode)',
       'log_s_s(region)', 'log_s_s(county)', 'iv_auto_tt', 'iv_transit_ivt',
       'iv_nonauto_tt', 'County_09001', 'County_09005', 'County_09009',
       'County_34003', 'County_34013', 'County_34017', 'County_34019',
       'County_34021', 'County_34023', 'County_34025', 'County_34027',
       'County_34029', 'County_34031', 'County_34035', 'County_34037',
       'County_34039', 'County_36005', 'County_36027', 'County_36047',
       'County_36059', 'County_36061-0', 'County_36061-1', 'Co

### NYMTC Data: Travel mode

In [69]:
mask = (data['destination_county']=='36061-1') & (data['origin_county']!='36061-1')
mask2 = (data['destination_county']=='36061-1') & (data['origin_county']=='36061-1') & (~data['mode'].isin(['walking','biking']))
data[mask|mask2]['Trip_num'].sum()

np.int64(2710719)

In [70]:
aa = data[mask|mask2].groupby('mode')['Trip_num'].sum()
aa

mode
biking        21769
carpool      358721
driving      256909
ondemand     211081
transit     1837895
walking       24344
Name: Trip_num, dtype: int64

In [139]:
# NYMTC Data (persons in thousand)
Person_By_Mode = {"AUTO, TAXI, VAN & TRUCK":800.40, "BUS":215.78, "SUBWAY":1494.26, "RAILROAD":296.57, 
                  "PASSENGER FERRY":46.11, "TRAM":5.15, "BICYCLE":42.01}

Person_By_Sector = {"NOTRH OF 60TH STREET":1071.24, "BROOKLYN":761.86, "QUEENS":570.03, "NEW JERSEY":471.56, "STATEN ISLAND":25.61}

In [140]:
np.array([x for x in Person_By_Mode.values()]).sum() - np.array([x for x in Person_By_Sector.values()]).sum()

np.float64(-0.019999999999527063)

### NYMTC Data: Section of Entry

In [141]:
weights = pd.read_csv('/Users/ryan/Documents/01.Research Projects/18.NYC Congestion Pricing/6.TR-A Revision/Replica Data Validation/Origin County – Sectors of Entry .csv',
                      dtype={'FIPs Code':'str'})[['Origin County','FIPs Code','Entry Sector','Weight']]
weights = weights._append({'Origin County':'CRZ','FIPs Code':'36061-1','Entry Sector':'60th Street','Weight':0.5},ignore_index=True)
weights = weights._append({'Origin County':'CRZ','FIPs Code':'36061-1','Entry Sector':'Brooklyn','Weight':0.2},ignore_index=True)
weights = weights._append({'Origin County':'CRZ','FIPs Code':'36061-1','Entry Sector':'Queens','Weight':0.2},ignore_index=True)
weights = weights._append({'Origin County':'CRZ','FIPs Code':'36061-1','Entry Sector':'New Jersey','Weight':0.1},ignore_index=True)

In [142]:
aa = data[mask|mask2]
aa['Trip_num'].sum()
aa['origin_county'] = aa['origin_county'].map(lambda x: '0'+x if len(x)==4 else x)
aa['origin_county'] = aa['origin_county'].map(lambda x: '36061' if x in ['36061-0'] else x)

In [143]:
weights.head()

,Origin County,FIPs Code,Entry Sector,Weight
0,Bronx,36005,60th Street,1.0
1,Kings,36047,Brooklyn,1.0
2,New York,36061,60th Street,1.0
3,Queens,36081,Queens,1.0
4,Richmond,36085,Brooklyn,0.5


In [144]:
merged = aa.merge(weights[['FIPs Code', 'Entry Sector', 'Weight']], 
                  left_on='origin_county', 
                  right_on='FIPs Code', 
                  how='left')

# Group by Entry Sector and sum Trip_num
merged['Trip_num'] = merged['Trip_num'] * merged['Weight']
result = merged.groupby('Entry Sector')['Trip_num'].sum().reset_index()
result

,Entry Sector,Trip_num
0,60th Street,982468.5
1,Brooklyn,725773.1
2,New Jersey,421040.8
3,Queens,581436.6


In [145]:
result['Trip_num'].sum()

np.float64(2710719.0)